In [1]:
import numpy as np
import pandas as pd
import random

In [7]:
data = pd.read_csv('steam-200k.csv', header = None, names = ['UserID', 'Game', 'Action', 'Hours', 'Not Needed'])

In [18]:
data.head()

,UserID,Game,Action,Hours,Not Needed,Hours_Played
0,151603712,The Elder Scrolls V Skyrim,purchase,1.0,0,0.0
1,151603712,The Elder Scrolls V Skyrim,play,273.0,0,273.0
2,151603712,Fallout 4,purchase,1.0,0,0.0
3,151603712,Fallout 4,play,87.0,0,87.0
4,151603712,Spore,purchase,1.0,0,0.0


In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 6 columns):
UserID          200000 non-null int64
Game            200000 non-null object
Action          200000 non-null object
Hours           200000 non-null float64
Not Needed      200000 non-null int64
Hours_Played    200000 non-null float32
dtypes: float32(1), float64(1), int64(2), object(2)
memory usage: 8.4+ MB


In [20]:
data.describe()

,UserID,Hours,Not Needed,Hours_Played
count,2.000000e+05,200000.000000,200000.0,200000.000000
mean,1.036559e+08,17.874384,0.0,17.226898
std,7.208074e+07,138.056952,0.0,138.073303
min,5.250000e+03,0.100000,0.0,0.000000
25%,4.738420e+07,1.000000,0.0,0.000000
50%,8.691201e+07,1.000000,0.0,0.000000
75%,1.542309e+08,1.300000,0.0,1.300000
max,3.099031e+08,11754.000000,0.0,11754.000000


In [21]:
data.shape

(200000, 6)

In [22]:
# 创建Hours_Played字段，替代原有的Action和Hours，0表示仅购买，大于0表示购买且游戏时长
data['Hours_Played'] = data['Hours'].astype('float32')
# 如果字段Action=purchase，并且Hours=1.0，将设置Hours_Played=0
data.loc[(data['Action'] == 'purchase') & (data['Hours'] == 1.0), 'Hours_Played'] = 0

In [23]:
data['Hours_Played']

0           0.0
1         273.0
2           0.0
3          87.0
4           0.0
          ...  
199995      1.5
199996      0.0
199997      1.5
199998      0.0
199999      1.4
Name: Hours_Played, Length: 200000, dtype: float32

In [24]:
data['Hours_Played'].describe()

count    200000.000000
mean         17.226898
std         138.073303
min           0.000000
25%           0.000000
50%           0.000000
75%           1.300000
max       11754.000000
Name: Hours_Played, dtype: float64

In [25]:
data.shape

(200000, 6)

In [27]:
# 对数据从小到大进行排序, df下标也会发生变化
data.UserID = data.UserID.astype('int')
data = data.sort_values(['UserID', 'Game', 'Hours_Played'], ascending=True)

In [28]:
data

,UserID,Game,Action,Hours,Not Needed,Hours_Played
65429,5250,Alien Swarm,purchase,1.0,0,0.0
65430,5250,Alien Swarm,play,4.9,0,4.9
65423,5250,Cities Skylines,purchase,1.0,0,0.0
65424,5250,Cities Skylines,play,144.0,0,144.0
65435,5250,Counter-Strike,purchase,1.0,0,0.0
...,...,...,...,...,...,...
170025,309812026,Robocraft,purchase,1.0,0,0.0
10221,309824202,Dota 2,purchase,1.0,0,0.0
10222,309824202,Dota 2,play,0.7,0,0.7
129084,309903146,Dota 2,purchase,1.0,0,0.0


In [29]:
# 删除重复项，并保留最后一项（因为最后一项为用户游戏时间，第一项为购买）
clean_data = data.drop_duplicates(['UserID', 'Game'], keep='last')
# 去掉不用的列
clean_data = clean_data.drop(['Action', 'Hours', 'Not Needed'], axis=1)

In [30]:
clean_data.shape

(128804, 3)

In [31]:
clean_data.head()

,UserID,Game,Hours_Played
65430,5250,Alien Swarm,4.9
65424,5250,Cities Skylines,144.0
65435,5250,Counter-Strike,0.0
65436,5250,Counter-Strike Source,0.0
65437,5250,Day of Defeat,0.0


In [32]:
# EDA
n_users = len(clean_data.UserID.unique())
n_games = len(clean_data.Game.unique())

In [33]:
n_users, n_games

(12393, 5155)

In [34]:
# 矩阵稀疏性
sparsity = clean_data.shape[0] / float(n_users * n_games)

In [35]:
sparsity

0.0020161564563957487